In [2]:
%load_ext autoreload
%autoreload 2

import os
import sys
sys.path.append("../../src")
import pandas as pd
import numpy as np
import synapseclient as sc
import utils.munging_utils as query

Welcome, aryton tediarjo!



In [11]:
df = query.get_walking_synapse_table(recordIds = ["507a9af9-4638-4549-9613-d6089d3b32c3"],
                               table_id = "syn12514611",
                               version = "V2")

Querying V2 Data


In [14]:
def get_fs(path):
    data = query.open_filepath(path)[1:]
    time_end = (data["timestamp"][-1:].iloc[0])
    N = data.shape[0]
    return N/time_end

In [23]:
data = query.open_filepath(df["walk_motion.json_pathfile"].iloc[0])

In [24]:
data.shape[0]/data["timestamp"].iloc[-1]

5.296694415279703

In [22]:
data

,eventAccuracy,sensor,sensorType,timestamp,timestampDate,unit,uptime,x,y,z
0,0,"{'mFifoMaxEventCount': 0, 'mFifoReservedEventC...",acceleration,0.000000,2019-11-18T22:52:32.559759523Z,NaN,599540.423790,NaN,NaN,NaN
1,0,NaN,acceleration,0.000000,NaN,g,599540.423790,-0.087891,0.029297,0.992188
2,0,NaN,acceleration,0.200012,NaN,g,599540.623802,-0.087891,0.031250,0.994141
3,0,NaN,acceleration,0.399994,NaN,g,599540.823784,-0.085938,0.031250,0.992188
4,0,NaN,acceleration,0.600342,NaN,g,599541.024132,-0.089844,0.037109,0.992188
5,0,NaN,acceleration,0.799988,NaN,g,599541.223778,-0.089844,0.037109,0.996094
6,0,NaN,acceleration,1.000000,NaN,g,599541.423790,-0.087891,0.037109,0.994141
7,0,NaN,acceleration,1.199982,NaN,g,599541.623772,-0.087891,0.035156,0.992188
8,0,NaN,acceleration,1.401123,NaN,g,599541.824913,-0.087891,0.035156,0.994141
9,0,NaN,acceleration,1.600525,NaN,g,599542.024315,-0.087891,0.035156,0.994141
